In [63]:
import numpy as np
import qutip as qt
from qutip import *
from qutip.qip.operations import hadamard_transform as H, cnot
from qutip.core.operators import identity as I, sigmax as X, sigmay as Y, sigmaz as Z
from qutip.core.states import bell_state

# Define basis state functions:
phi_l_plus = lambda l: (tensor(basis(2, 0), basis(2, 0)) + l * tensor(basis(2, 1), basis(2, 1))) / np.sqrt(1 + np.abs(l)**2)
phi_l_minus = lambda l: (l.conjugate() * tensor(basis(2, 0), basis(2, 0)) - tensor(basis(2, 1), basis(2, 1))) / np.sqrt(1 + np.abs(l)**2)
psi_p_plus = lambda p: (tensor(basis(2, 0), basis(2, 1)) + p * tensor(basis(2, 1), basis(2, 0))) / np.sqrt(1 + np.abs(p)**2)
psi_p_minus = lambda p: (p.conjugate() * tensor(basis(2, 0), basis(2, 1)) - tensor(basis(2, 1), basis(2, 0))) / np.sqrt(1 + np.abs(p)**2)


def ptp_resources(n):
    psi0 = rand_ket(2)  #unknown state
    phi_AB = tensor(basis(2, 0), basis(2, 0)) + n * tensor(basis(2, 1), basis(2, 1)) #entangled state between Alice and Bob
    phi_AB = phi_AB / phi_AB.norm()
    l = n
    p = n.conjugate()
    basis_states = [phi_l_plus(l), phi_l_minus(l), psi_p_plus(p), psi_p_minus(p)] #set of basis set to make the 
    return psi0, phi_AB, basis_states

def ptp_measurement(psi0, phi_AB, basis, corrs, labels):
    psi_0AB = tensor(psi0, phi_AB)
    target = psi0 * psi0.dag()
    res = []
    for i, lab in enumerate(labels):
        proj = basis[i] * basis[i].dag()
        measure_op = tensor(proj, I(2))
        psi_projected = measure_op * psi_0AB
        prob = expect(measure_op, psi_0AB)
        if prob > 1e-10:
            psi_normalized = psi_projected / np.sqrt(prob)
            dm = psi_normalized.ptrace([2])  # Bob’s uncorrected state
            U = corrs[lab]
            rho_bob_corr = U * dm * U.dag()  # Bob’s corrected state
            fid = qt.metrics.fidelity(rho_bob_corr, target)
            # Store uncorrected state for debugging
            res.append((lab, prob, fid, dm))
    return res

def main():
    # Use a fixed n to ensure non-maximal entanglement
    n = np.random.rand() + 1j * np.random.rand()  # |n| < 1 for clear distinction
    psi0, phi_AB, basis = ptp_resources(n)
    
    corrs = {
        'phi_plus': I(2),    # No correction
        'phi_minus': Z(),    # σ_z
        'psi_plus': X(),     # σ_x
        'psi_minus': I(2)    # No correction
    }
    labels = ['phi_plus', 'phi_minus', 'psi_plus', 'psi_minus']
    
    results = ptp_measurement(psi0, phi_AB, basis, corrs, labels)
    
    print(f"Resource state parameter n = {n:.4f}")
    print(f"Original state: \n{psi0}")
    print("\nTeleportation Outcomes:")
    for lab, prob, fid, dm in results:
        print(f"Outcome: {lab}")
        print(f"  Probability: {prob:.4f}")
        print(f"  Fidelity: {fid:.4f}")
        print(f"  Bob’s uncorrected state: \n{dm}\n")
    
    n_abs_sq = np.abs(n)**2
    expected_p_succ = 2 * n_abs_sq / (1 + n_abs_sq)**2
    success_prob = sum(prob for lab, prob, _, _ in results if lab in ['phi_minus', 'psi_plus'])
    print(f"Expected success probability (paper): {expected_p_succ:.4f}")
    print(f"Computed success probability (phi_minus + psi_plus): {success_prob:.4f}")

if __name__ == "__main__":
    main()

Resource state parameter n = 0.7496+0.0188j
Original state: 
Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.54109796+0.52913107j]
 [-0.62548842-0.1897302j ]]

Teleportation Outcomes:
Outcome: phi_plus
  Probability: 0.2900
  Fidelity: 0.9663
  Bob’s uncorrected state: 
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.80916802+0.j         -0.34860397-0.18135712j]
 [-0.34860397+0.18135712j  0.19083198+0.j        ]]

Outcome: phi_minus
  Probability: 0.2304
  Fidelity: 1.0000
  Bob’s uncorrected state: 
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.57276668+0.j         0.43884265+0.22830273j]
 [0.43884265-0.22830273j 0.42723332+0.j        ]]

Outcome: psi_plus
  Probability: 0.2304
  Fidelity: 1.0000
  Bob’s uncorrected state: 
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.42723332+0.j  

In [73]:
import numpy as np
import qutip as qt
from qutip import *
from qutip.qip.operations import hadamard_transform as H, cnot
from qutip.core.operators import identity as I, sigmax as X, sigmay as Y, sigmaz as Z
from qutip.core.states import bell_state

# Define basis state functions for Charlie’s measurement (non-maximally entangled basis)
phi_l_plus = lambda l: (tensor(basis(2, 0), basis(2, 0)) + l * tensor(basis(2, 1), basis(2, 1))) / np.sqrt(1 + np.abs(l)**2)
phi_l_minus = lambda l: (l.conjugate() * tensor(basis(2, 0), basis(2, 0)) - tensor(basis(2, 1), basis(2, 1))) / np.sqrt(1 + np.abs(l)**2)
psi_p_plus = lambda p: (tensor(basis(2, 0), basis(2, 1)) + p * tensor(basis(2, 1), basis(2, 0))) / np.sqrt(1 + np.abs(p)**2)
psi_p_minus = lambda p: (p.conjugate() * tensor(basis(2, 0), basis(2, 1)) - tensor(basis(2, 1), basis(2, 0))) / np.sqrt(1 + np.abs(p)**2)

def es_resources(n1, n2):
    # A-B : |Φ_AB> = (|00> + n1|11>)/norm
    psi_AB = tensor(basis(2, 0), basis(2, 0)) + n1 * tensor(basis(2, 1), basis(2, 1))
    psi_AB = psi_AB / psi_AB.norm()
    
    # C-D : |Φ_CD> = (|00> + n2|11>)/norm
    psi_CD = tensor(basis(2, 0), basis(2, 0)) + n2 * tensor(basis(2, 1), basis(2, 1))
    psi_CD = psi_CD / psi_CD.norm()
    # parameters for making measurement on BC 
    l = n1
    p = n1.conjugate()
    basis_states = [phi_l_plus(l), phi_l_minus(l), psi_p_plus(p), psi_p_minus(p)]
    return psi_AB, psi_CD, basis_states

def swapping_measurement(psi_ABCD, basis_states, corrs, labels, n1):
    # Target state: |Φ_AD> = (|00> + n1|11>)/norm
    phi_AD_target = tensor(basis(2, 0), basis(2, 0)) + n1 * tensor(basis(2, 1), basis(2, 1))
    phi_AD_target = phi_AD_target / phi_AD_target.norm()
    target = phi_AD_target * phi_AD_target.dag()
    
    res = []
    # Measure qubits B and C
    for i, lab in enumerate(labels):
        proj_BC = basis_states[i] * basis_states[i].dag()  # Projector on B and C
        measure_op = tensor(qeye(2), proj_BC, qeye(2))     # Identity on A and D, projector on B-C
        psi_projected = measure_op * psi_ABCD
        prob = expect(measure_op, psi_ABCD)  # Use expect for probability
        if prob > 1e-10:
            psi_norm = psi_projected / np.sqrt(prob)
            rho_AD = psi_norm.ptrace([0, 3])  # Extract state of qubits A and D
            
            # Apply correction on qubit D (second qubit in A-D system)
            U = corrs[lab]
            U_ext = tensor(qeye(2), U)  # Identity on A, correction on D
            rho_AD_corr = U_ext * rho_AD * U_ext.dag()
            
            # Compute concurrence and fidelity
            con = concurrence(rho_AD_corr)
            fid = qt.metrics.fidelity(rho_AD_corr, target)
            res.append((lab, prob, con, fid))
    return res

def main():
    n1 = np.random.rand() + 1j * np.random.rand()  # A-B pair
    n2 = np.random.rand() + 1j * np.random.rand()  # C-D pair
    # Generate resources
    psi_AB, psi_CD, basis_states = es_resources(n1, n2)
    psi_ABCD = tensor(psi_AB, psi_CD) 
    corrs = {
        'phi_plus': I(2),    # No correction (fidelity < 1)
        'phi_minus': Z(),    # σ_z for unit fidelity
        'psi_plus': X(),     # σ_x for unit fidelity
        'psi_minus': I(2)    # No correction (fidelity < 1)
    }
    labels = ['phi_plus', 'phi_minus', 'psi_plus', 'psi_minus']
    results = swapping_measurement(psi_ABCD, basis_states, corrs, labels, n1)
    
    # Print results
    print(f"Entanglement parameters: n1 = {n1:.4f}, n2 = {n2:.4f}")
    print(f"Initial A-B state: \n{psi_AB}")
    print(f"Initial C-D state: \n{psi_CD}")
    print(f"Target A-D entangled state: \n{(tensor(basis(2, 0), basis(2, 0)) + n1 * tensor(basis(2, 1), basis(2, 1))).unit()}")
    print("\nEntanglement Swapping Outcomes:")
    for lab, prob, con, fid in results:
        print(f"Outcome: {lab}")
        print(f"  Probability: {prob:.4f}")
        print(f"  Fidelity with target: {fid:.4f}")
        print(f"  Concurrence: {con:.4f}\n")
    
    # Verify success probability
    n1_abs_sq = np.abs(n1)**2
    expected_p_succ = 2 * n1_abs_sq / (1 + n1_abs_sq)**2
    success_prob = sum(prob for lab, prob, _, _ in results if lab in ['phi_minus', 'psi_plus'])
    print(f"Expected success probability (paper): {expected_p_succ:.4f}")
    print(f"Computed success probability (phi_minus + psi_plus): {success_prob:.4f}")

if __name__ == "__main__":
    main()

Entanglement parameters: n1 = 0.6235+0.2561j, n2 = 0.3004+0.4483j
Initial A-B state: 
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.8292042 +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.51704304+0.21233673j]]
Initial C-D state: 
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.88002366+0.j       ]
 [0.        +0.j       ]
 [0.        +0.j       ]
 [0.26437238+0.3945448j]]
Target A-D entangled state: 
Quantum object: dims=[[2, 2], [1, 1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.8292042 +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.51704304+0.21233673j]]

Entanglement Swapping Outcomes:
Outcome: phi_plus
  Probability: 0.3881
  Fidelity with target: 0.9189
  Concurrence: 0.4626

Outcome: phi_minus
  Probability: 0.2148
  Fidelity with target: 0.9616
  Concurrence: 0.8359

Outcome: psi_plus
  Probability: 0.1822
  Fidelity with target: 0